# Analysis of Transfer Events of a Token Contract

The Aim of this exercise is to query an ethereum node and find out all the transfer events from any `starting block` to an `ending_block`. We consider not a very large `block_range`(`starting_block` - `ending_block`) as we can query at max `10,000` events in a single request to the node.

Here we write the code such that this should be able to take the contract address of any token and return the analysis. All we need to do is change the `contract_address` variable and the rest should be the same.

## Setup

In [1]:
## Import Libraries

from web3 import Web3
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Set up Infura Node

import getpass

# Infura API setup
API_KEY = getpass.getpass()
infura_url = f"https://mainnet.infura.io/v3/{API_KEY}"
web3 = Web3(Web3.HTTPProvider(infura_url))

## Parse Token Details and Transfer Events from Token Contract

In [ ]:
# Token contract address
contract_address = "0xBB0E17EF65F82Ab018d8EDd776e8DD940327B28b" ## Axie Infinity Contract

# Contract ABI for Transfer Events, Token Decimals and Contract Name
contract_abi = [
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "internalType": "address",
                "name": "_from",
                "type": "address"
            },
            {
                "indexed": True,
                "internalType": "address",
                "name": "_to",
                "type": "address"
            },
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "_value",
                "type": "uint256"
            }
        ],
        "name": "Transfer",
        "type": "event"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [
            {
                "internalType": "uint8",
                "name": "",
                "type": "uint8"
            }
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            }
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }
]

# Set the range of blocks to scan for events

from_block = 17351837 # Replace with desired starting block
to_block = 17361837 # Replace with  desired ending block

# Set up contract instance
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

# Get the name of the contract
name = contract.functions.name().call()

# Get the no of decimals for the token
decimals = contract.functions.decimals().call()

# Get Transfer events
events = contract.events.Transfer.getLogs(fromBlock=from_block, toBlock=to_block)

In [ ]:
def find_block_timestamp(blockNumber):
    '''Finds block date for a given block number. Only an estimation data as caclulating exact block date
    from node requires multiple requests. 
    
    Using block number 17351837(https://etherscan.io/block/17351837) for reference.
    
    Block timestamp for block 17351837 = 1685207(`May-27-2023 05:19:47 PM +UTC`)
    
    Hence, Block timestamp for block x = AVG_BLOCK_TIME * (blockNumber - 17351837) + 1685207
    '''
    
    AVG_BLOCK_TIME = 12
    return pd.to_datetime(AVG_BLOCK_TIME * (blockNumber - 17351837) + 1685207987, unit='s').date()

find_block_timestamp(17351837)

In [ ]:
events_pd = pd.DataFrame(events)

def parse_args(row):
    row['from_address'] = row['args'].get('_from')
    row['to_address'] = row['args'].get('_to')
    row['value'] = int(row['args'].get('_value'))/pow(10,decimals)
    row['transactionHash'] = row['transactionHash'].hex()
    row['blockHash'] = row['blockHash'].hex()
    row['blockDate'] = find_block_timestamp(row['blockNumber'])
    return row

events_pd = events_pd.apply(parse_args, axis=1)
events_pd

## Some basic EDA on Daily Transfer Events

### Plots on Daily Transaction Count and Transaction Values

In [ ]:
events_agg_by_date = events_pd.groupby('blockDate')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

plt.title(f'Daily Aggregated plots for \ntoken: {name}, \naddress: {contract_address}')

events_agg_by_date.transactionIndex.nunique().plot.bar(ax = ax1)
ax1.set_title(f'Tranfer Counts')

events_agg_by_date.value.sum().plot.bar(ax = ax2)
ax2.set_title(f'Transfer Value')

plt.show()

## Some basic EDA on Events aggregated by user

In [ ]:
tokens_sent_per_user = events_pd.groupby('from_address').value.sum().reset_index().rename(
    columns={'from_address':'address'}
)
tokens_received_per_user = events_pd.groupby('to_address').value.sum().reset_index().rename(
    columns={'to_address':'address'}
) 

events_agg_by_address = pd.merge(
    tokens_sent_per_user, 
    tokens_received_per_user, 
    how='outer', 
    on='address', 
    suffixes=['_sent','_received']
).fillna(0)

events_agg_by_address['balance'] = events_agg_by_address['value_received'] - events_agg_by_address['value_sent'] 

### Top 5 balance holders

In [ ]:
top_balance_holders = events_agg_by_address.sort_values('balance', ascending=False).head()
display(top_balance_holders)

### Average balance per token holder

In [ ]:
print("Average Value sent per user: ", np.mean(events_agg_by_address['value_sent']))